# Extract Features

1. Read csv files and load as dfs
2. Combine dfs
3. Get semantic cosine similarity

In [1]:
import os, sys

import pandas as pd

from tqdm import tqdm

# Get the current working directory of the notebook
notebook_dir = os.getcwd()
print(notebook_dir)
# Add the parent directory to the system path
sys.path.append(os.path.join(notebook_dir, '../'))

import log_files
from log_files import LogData
from data_processing import DataProcessing
from feature_extraction import SpacyFeatureExtraction
from text_generation_models import TextGenerationModelFactory

/Users/detraviousjamaribrinkley/Documents/Development/research_labs/uf_ds/predictions/prediction_correctness_experiments


In [2]:
pd.set_option('max_colwidth', 800)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

## Predictions

- Use the structure from `1-generate_predictions-all_domains.ipynb`

In [3]:
log_file_path = "data/prediction_logs"
predictions = True
predictions_df = log_files.read_data(notebook_dir, log_file_path, predictions)
predictions_df.head(7)

Start logging batch
log_directory: /Users/detraviousjamaribrinkley/Documents/Development/research_labs/uf_ds/predictions/prediction_correctness_experiments/../data/prediction_logs
save_batch_directory: /Users/detraviousjamaribrinkley/Documents/Development/research_labs/uf_ds/predictions/prediction_correctness_experiments/../data/prediction_logs/batch_1-prediction
CSV to DF
Load saved csv: /Users/detraviousjamaribrinkley/Documents/Development/research_labs/uf_ds/predictions/prediction_correctness_experiments/../data/prediction_logs/batch_1-prediction/batch_1-from_df.csv
save_batch_directory: /Users/detraviousjamaribrinkley/Documents/Development/research_labs/uf_ds/predictions/prediction_correctness_experiments/../data/prediction_logs/batch_2-prediction
CSV to DF
Load saved csv: /Users/detraviousjamaribrinkley/Documents/Development/research_labs/uf_ds/predictions/prediction_correctness_experiments/../data/prediction_logs/batch_2-prediction/batch_2-from_df.csv
save_batch_directory: /Users

,Base Sentence,Sentence Label,Domain,Model Name,API Name,Batch ID,Template Number
0,JPMorgan Chase forecasts that the net profit at Amazon potentially decrease in Q3 of 2027.,1,finance,llama-3.1-70b-instruct,NAVI_GATOR,0,1
1,"On August 21, 2024, Bank of America speculates the revenue at Microsoft will likely increase.",1,finance,llama-3.1-70b-instruct,NAVI_GATOR,0,2
2,"Citigroup predicts on 2024-08-21, the operating income at Alphabet may rise.",1,finance,llama-3.1-70b-instruct,NAVI_GATOR,0,3
3,"According to Goldman Sachs, the research and development expenses at Facebook would fall in 2025.",1,finance,llama-3.1-70b-instruct,NAVI_GATOR,0,4
4,"In 21 August 2024, Morgan Stanley envisions that the gross profit at Johnson & Johnson has some probability to remain stable.",1,finance,llama-3.1-70b-instruct,NAVI_GATOR,0,5
5,"The stock price at Visa should stay same in Q2 of 2026, according to Wells Fargo.",1,finance,llama-3.1-70b-instruct,NAVI_GATOR,0,6
6,JPMorgan forecasts that the revenue at Microsoft potentially decrease in Q3 of 2027.,1,finance,llama-3.3-70b-instruct,NAVI_GATOR,0,1


## Observations

In [4]:
log_file_path = "data/observation_logs"
predictions = False
observations_df = log_files.read_data(notebook_dir, log_file_path, predictions)
observations_df.head(7)

Start logging batch
log_directory: /Users/detraviousjamaribrinkley/Documents/Development/research_labs/uf_ds/predictions/prediction_correctness_experiments/../data/observation_logs
save_batch_directory: /Users/detraviousjamaribrinkley/Documents/Development/research_labs/uf_ds/predictions/prediction_correctness_experiments/../data/observation_logs/batch_1-observation
CSV to DF
Load saved csv: /Users/detraviousjamaribrinkley/Documents/Development/research_labs/uf_ds/predictions/prediction_correctness_experiments/../data/observation_logs/batch_1-observation/batch_1-from_df.csv
save_batch_directory: /Users/detraviousjamaribrinkley/Documents/Development/research_labs/uf_ds/predictions/prediction_correctness_experiments/../data/observation_logs/batch_2-observation
CSV to DF
Load saved csv: /Users/detraviousjamaribrinkley/Documents/Development/research_labs/uf_ds/predictions/prediction_correctness_experiments/../data/observation_logs/batch_2-observation/batch_2-from_df.csv
save_batch_director

,Base Sentence,Sentence Label,Domain,Model Name,API Name,Batch ID,Template Number
0,The financial analyst at Goldman Sachs observed that the operating income at Tesla had increased in the first quarter of 2024.,0,finance,llama-3.1-70b-instruct,NAVI_GATOR,0,1
1,"On 2024-08-20 to 2025-08-20, Morgan Stanley speculates the stock price at Amazon will likely rise.",0,finance,llama-3.1-70b-instruct,NAVI_GATOR,0,2
2,"A young investor predicts on 2025-03-15, the S&P 500 index may rise.",0,finance,llama-3.1-70b-instruct,NAVI_GATOR,0,3
3,"According to Bank of America, the net profit at Microsoft would fall in the second quarter of 2026.",0,finance,llama-3.1-70b-instruct,NAVI_GATOR,0,4
4,"In 2027-01-01 to 2027-12-31, Wells Fargo envisions that the interest rates at the Federal Reserve have some probability to remain stable.",0,finance,llama-3.1-70b-instruct,NAVI_GATOR,0,5
5,"The trading volume at Apple should stay same in the fourth quarter of 2025, according to a financial expert at JPMorgan Chase.",0,finance,llama-3.1-70b-instruct,NAVI_GATOR,0,6
6,JPMorgan observed that the net profit at Microsoft had risen in September 2023.,0,finance,llama-3.3-70b-instruct,NAVI_GATOR,0,1


In [5]:
predictions = DataProcessing.df_to_list(predictions_df, "Base Sentence")
observations = DataProcessing.df_to_list(observations_df, "Base Sentence")

In [6]:
tgmf = TextGenerationModelFactory()

# Groq Cloud (https://console.groq.com/docs/overview)
gemma_29b_generation_model = tgmf.create_instance('gemma2-9b-it') 
llama_318b_instant_generation_model = tgmf.create_instance('llama-3.1-8b-instant') 
llama_3370b_versatile_generation_model = tgmf.create_instance('llama-3.3-70b-versatile')  
# llama_guard_4_12b_generation_model = tgmf.create_instance('meta-llama/llama-guard-4-12b')  

models = [gemma_29b_generation_model, llama_3370b_versatile_generation_model]

In [7]:
def llm_certifier(predictions: list, observations: list):

    preds_obs_labels = []
    for prediction in tqdm(predictions[:3]):
        
        for observation in observations[:3]:
            prompt = f"Return if the observation ({observation}) certifies this prediction ({prediction}). Let the return label be either certify, not certify, neither and nothing more."
            print(f"Prompt: {prompt}")

            print()
            # # df = pd.DataFrame(columns=["Model", "Prompt", "Response"])
            # model_certify = {}
            for model in models:  
                input_prompt = model.user(prompt)
                # print(input_prompt)  
                
                raw_text = model.chat_completion([input_prompt])
                # print(raw_text) 
                # print() 
                output = []
                for line in raw_text.split("\n"):
                    if line.strip():  # Skip empty lines
                        output.append(line.strip())
                pred_ob_label = (prediction, observation, output)
                preds_obs_labels.append(pred_ob_label)
    return preds_obs_labels

In [8]:
certifier_predictions = llm_certifier(predictions, observations)
certifier_predictions

  0%|          | 0/3 [00:00<?, ?it/s]

Prompt: Return if the observation (The financial analyst at Goldman Sachs observed that the operating income at Tesla had increased in the first quarter of 2024.) certifies this prediction (JPMorgan Chase forecasts that the net profit at Amazon potentially decrease in Q3 of 2027.). Let the return label be either certify, not certify, neither and nothing more.

Prompt: Return if the observation (On 2024-08-20 to 2025-08-20, Morgan Stanley speculates the stock price at Amazon will likely rise.) certifies this prediction (JPMorgan Chase forecasts that the net profit at Amazon potentially decrease in Q3 of 2027.). Let the return label be either certify, not certify, neither and nothing more.

Prompt: Return if the observation (A young investor predicts on 2025-03-15, the S&P 500 index may rise.) certifies this prediction (JPMorgan Chase forecasts that the net profit at Amazon potentially decrease in Q3 of 2027.). Let the return label be either certify, not certify, neither and nothing more

 33%|███▎      | 1/3 [00:02<00:04,  2.11s/it]

Prompt: Return if the observation (The financial analyst at Goldman Sachs observed that the operating income at Tesla had increased in the first quarter of 2024.) certifies this prediction (On August 21, 2024, Bank of America speculates the revenue at Microsoft will likely increase.). Let the return label be either certify, not certify, neither and nothing more.

Prompt: Return if the observation (On 2024-08-20 to 2025-08-20, Morgan Stanley speculates the stock price at Amazon will likely rise.) certifies this prediction (On August 21, 2024, Bank of America speculates the revenue at Microsoft will likely increase.). Let the return label be either certify, not certify, neither and nothing more.

Prompt: Return if the observation (A young investor predicts on 2025-03-15, the S&P 500 index may rise.) certifies this prediction (On August 21, 2024, Bank of America speculates the revenue at Microsoft will likely increase.). Let the return label be either certify, not certify, neither and not

 67%|██████▋   | 2/3 [00:04<00:02,  2.02s/it]

Prompt: Return if the observation (The financial analyst at Goldman Sachs observed that the operating income at Tesla had increased in the first quarter of 2024.) certifies this prediction (Citigroup predicts on 2024-08-21, the operating income at Alphabet may rise.). Let the return label be either certify, not certify, neither and nothing more.

Prompt: Return if the observation (On 2024-08-20 to 2025-08-20, Morgan Stanley speculates the stock price at Amazon will likely rise.) certifies this prediction (Citigroup predicts on 2024-08-21, the operating income at Alphabet may rise.). Let the return label be either certify, not certify, neither and nothing more.

Prompt: Return if the observation (A young investor predicts on 2025-03-15, the S&P 500 index may rise.) certifies this prediction (Citigroup predicts on 2024-08-21, the operating income at Alphabet may rise.). Let the return label be either certify, not certify, neither and nothing more.



100%|██████████| 3/3 [00:06<00:00,  2.04s/it]


[('JPMorgan Chase forecasts that the net profit at Amazon potentially decrease in Q3 of 2027.',
  'The financial analyst at Goldman Sachs observed that the operating income at Tesla had increased in the first quarter of 2024.',
  ['neither']),
 ('JPMorgan Chase forecasts that the net profit at Amazon potentially decrease in Q3 of 2027.',
  'The financial analyst at Goldman Sachs observed that the operating income at Tesla had increased in the first quarter of 2024.',
  ["Not certify. The observation is about Tesla's operating income, while the prediction is about Amazon's net profit. They are two different companies and metrics, so the observation does not provide any information that would certify the prediction."]),
 ('JPMorgan Chase forecasts that the net profit at Amazon potentially decrease in Q3 of 2027.',
  'On 2024-08-20 to 2025-08-20, Morgan Stanley speculates the stock price at Amazon will likely rise.',
  ['neither']),
 ('JPMorgan Chase forecasts that the net profit at Amazo

In [14]:
df = pd.DataFrame(certifier_predictions, columns=['Prediction Sentence', 'Observation Sentence', 'Label'])
df


,Prediction Sentence,Observation Sentence,Label
0,JPMorgan Chase forecasts that the net profit at Amazon potentially decrease in Q3 of 2027.,The financial analyst at Goldman Sachs observed that the operating income at Tesla had increased in the first quarter of 2024.,[neither]
1,JPMorgan Chase forecasts that the net profit at Amazon potentially decrease in Q3 of 2027.,The financial analyst at Goldman Sachs observed that the operating income at Tesla had increased in the first quarter of 2024.,"[Not certify. The observation is about Tesla's operating income, while the prediction is about Amazon's net profit. They are two different companies and metrics, so the observation does not provide any information that would certify the prediction.]"
2,JPMorgan Chase forecasts that the net profit at Amazon potentially decrease in Q3 of 2027.,"On 2024-08-20 to 2025-08-20, Morgan Stanley speculates the stock price at Amazon will likely rise.",[neither]
3,JPMorgan Chase forecasts that the net profit at Amazon potentially decrease in Q3 of 2027.,"On 2024-08-20 to 2025-08-20, Morgan Stanley speculates the stock price at Amazon will likely rise.",[not certify.]
4,JPMorgan Chase forecasts that the net profit at Amazon potentially decrease in Q3 of 2027.,"A young investor predicts on 2025-03-15, the S&P 500 index may rise.",[neither]
5,JPMorgan Chase forecasts that the net profit at Amazon potentially decrease in Q3 of 2027.,"A young investor predicts on 2025-03-15, the S&P 500 index may rise.",[not certify.]
6,"On August 21, 2024, Bank of America speculates the revenue at Microsoft will likely increase.",The financial analyst at Goldman Sachs observed that the operating income at Tesla had increased in the first quarter of 2024.,[neither]
7,"On August 21, 2024, Bank of America speculates the revenue at Microsoft will likely increase.",The financial analyst at Goldman Sachs observed that the operating income at Tesla had increased in the first quarter of 2024.,[not certify.]
8,"On August 21, 2024, Bank of America speculates the revenue at Microsoft will likely increase.","On 2024-08-20 to 2025-08-20, Morgan Stanley speculates the stock price at Amazon will likely rise.",[neither]
9,"On August 21, 2024, Bank of America speculates the revenue at Microsoft will likely increase.","On 2024-08-20 to 2025-08-20, Morgan Stanley speculates the stock price at Amazon will likely rise.","[Not certify. The observation is about Amazon's stock price, while the prediction is about Microsoft's revenue. They are two different companies and metrics, so the observation does not certify the prediction.]"


In [ ]:


# # https://huggingface.co/mistralai/Mistral-7B-Instruct-v0.3
# from transformers import pipeline

In [ ]:

# prompt = f"Return top 5 list of the observations ({observations}) that certify this prediction ({predictions[0]})?"

# messages = [
#     {"role": "system", "content": prompt},
#     {"role": "user", "content": prompt},
# ]
# chatbot = pipeline("text-generation", model="mistralai/Mistral-7B-Instruct-v0.3")
# chatbot(messages)